# Data Preprocessing/cleaning up

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
file_path = "/content/drive/MyDrive/Big Data Final Project/Final CSVs"
#file_path = 'https://github.com/emdoehring/FinalProject/tree/9740336a6b27d4dd272346bcb02f115398c65117/Final%20CSVs'

In [ ]:
!pip install tensorflow

In [3]:
#IMPORTS:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import random
import requests as rq
import sys
import io
import time
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard
%load_ext tensorboard


# Data Preprocessing

In [4]:
df_lumineers = pd.read_csv('/content/drive/MyDrive/Big Data Final Project/Final CSVs/The Lumineers Lyrics.csv')

print(df_lumineers.to_string())


                               Unnamed: 0         Artist                                                              Album                            Release Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [5]:
#CONVERTING SPECIFIC LYRICS COLUMN INTO ITS OWN TXT FILE

df_lumineers['Lyrics'].to_csv(r'lumineers_data.txt', header=None, index=None, sep='\n', mode='a')
pd.read_csv('lumineers_data.txt')

,"Ah, ah, when I was younger\nI, I should've known better\nAnd I can't feel no remorse\nAnd you don't feel nothing back\n\nI, I got a new girlfriend here\nFeels like he's on top\nAnd I don't feel no remorse\nAnd you can't see past my blindness\n\nOh, Ophelia\nYou've been on my mind, girl, since the flood\nOh, Ophelia\nHeaven help the fool who falls in love\n\nI, I got a little paycheck\nYou got big plans and you gotta move Whoo!)\nAnd I don't feel nothing at all\nAnd you can't feel nothing small\n\nHoney, I love you""\nThat's all she wrote\n\nOh, Ophelia\nYou've been on my mind, girl, like a drug\nOh, Ophelia\nHeaven help the fool who falls in love\nOh, Ophelia\nYou've been on my mind, girl, since the flood\nOh, Ophelia\nHeaven help the fool who falls in love\nOh, Ophelia\nYou've been on my mind, girl, like a drug\nOh, Ophelia\nHeaven help the fool who falls in love"
0,"I was Cleopatra, I was young and an actress\nW..."
1,Ho! Hey!\nHo! Hey!\nHo!\n\nI've been tryin' to...
2,"Pack yourself a toothbrush, dear\nPack yoursel..."
3,She'll lie and steal and cheat\nAnd beg you fr...
4,"When you left this town, with your windows dow..."
...,...
75,"When you left this town, with your windows dow..."
76,"Ah, ah, when I was younger\nI, I should've kno..."
77,"It's not the words you say, but how you say it..."
78,"Gloria, I smell it on your breath\nGloria, boo..."


In [6]:
lyrics_path = 'lumineers_data.txt'
text = open(lyrics_path, 'rb').read().decode(encoding='utf-8')

In [7]:
stopChars = [',','(',')','.','-','[',']','"']

In [10]:
def preprocessText(text):
  text = text.replace('\n', ' ').replace('\t','')
  processedText = text.lower()
  for char in stopChars:
    processedText = processedText.replace(char,' ')
  return processedText

In [8]:
def WordsToList(lyrics_text):
  word_list = [w for w in lyrics_text.split(' ')]
  word_list = [i for i in word_list if i] #removing empty strings from list
  return word_list

In [11]:
preprocessText(text)

" ah  ah  when i was younger i  i should've known better and i can't feel no remorse and you don't feel nothing back  i  i got a new girlfriend here feels like he's on top and i don't feel no remorse and you can't see past my blindness  oh  ophelia you've been on my mind  girl  since the flood oh  ophelia heaven help the fool who falls in love  i  i got a little paycheck you got big plans and you gotta move whoo!  and i don't feel nothing at all and you can't feel nothing small  honey  i love you   that's all she wrote  oh  ophelia you've been on my mind  girl  like a drug oh  ophelia heaven help the fool who falls in love oh  ophelia you've been on my mind  girl  since the flood oh  ophelia heaven help the fool who falls in love oh  ophelia you've been on my mind  girl  like a drug oh  ophelia heaven help the fool who falls in love   i was cleopatra  i was young and an actress when you knelt by my mattress  and asked for my hand but i was sad you asked it  as i laid in a black dress w

In [12]:
lyrics_path = 'lumineers_data.txt'
text = open(lyrics_path, 'rb').read().decode(encoding='utf-8')
text = preprocessText(text)
lyrics_words = WordsToList(text)
map(str.strip, lyrics_words)


vocab = sorted(set(lyrics_words))
print('Lyrics length (in words):', len(lyrics_words))
print('Unique words in Lyrics text: {}'.format(len(vocab)))
word2idx = {u: i for i, u in enumerate(vocab)}
idx2words = np.array(vocab)
word_as_int = np.array([word2idx[c] for c in lyrics_words])
# The maximum length sentence we want for a single input in words
seqLength = 10
examples_per_epoch = len(lyrics_words)//(seqLength + 1)

Lyrics length (in words): 15455
Unique words in Lyrics text: 1852


In [13]:
wordDataset = tf.data.Dataset.from_tensor_slices(word_as_int)

sequencesOfWords = wordDataset.batch(seqLength + 1, drop_remainder=True) # generating batches of 10 words each

In [14]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequencesOfWords.map(split_input_target)

In [15]:
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [16]:
seqlen = 10
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

x = np.zeros((len(sentences), seqlen, len(chars)), dtype= bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype= bool)
for i, sentence in enumerate(sentences):

    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


# Training and Running Model

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)


def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #

def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)



    for diversity in [0.5]:
        print('----- diversity:', diversity)

        generated = ''
        # sentence = text[start_index: start_index + seqlen]
        sentence = seed
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(5000):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.


            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

seed = 'gloria'

model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback])





Epoch 1/50
60/61 [============================>.] - ETA: 0s - loss: 2.6423 - categorical_crossentropy: 2.6423 - accuracy: 0.2607
----- Generating text after Epoch: 0
----- diversity: 0.5
----- Generating with seed: "gloria"
gloria                                                                                                                                            t                         t                      h                         t                                        t   t                                                o     ot           t                           t                                                    t             i                       t                 t                                                                             et                                                        a    et                                                                       teo                                                                                        

In [ ]:
model.save('lyrics_model_v4.keras')
